In [27]:
from collections import deque
from pydantic import BaseModel
from typing import List, Optional, Dict
from datetime import datetime
from decimal import Decimal
import random

# Models

In [28]:
class Company(BaseModel):
    id: int
    name: str

class Stock(BaseModel):
    id: int
    company: Company
    ticker: str
    price: float
    open_price: float
    high_price: float
    low_price: float
    close_price: float
    partial_share: float
    complete_share: int
    timestamp: datetime
    volatility: float
    liquidity: float

class StockPriceHistory(BaseModel):
    id: int
    stock: Stock
    open_price: float
    high_price: float
    low_price: float
    close_price: float
    timestamp: datetime
    volatility: float
    liquidity: float


class UserProfile(BaseModel):
    id: int
    user_id: int
    team_id: Optional[int]
    timestamp: datetime


class Order(BaseModel):
    id: int
    user: UserProfile
    scenario_id: int
    stock: Stock
    quantity: int
    price: float
    transaction_type: str
    timestamp: datetime

class TransactionHistory(BaseModel):
    id: int
    orders: List[Order]

class Portfolio(BaseModel):
    id: int
    owner: UserProfile
    balance: float
    stocks: List[Stock]
    transactions: List[TransactionHistory]

class StockPortfolio(BaseModel):
    id: int
    stock: Stock
    portfolio: Portfolio
    quantity: int

# Database

In [29]:
companies = {1: Company(id=1, name="Apple Inc")}
stocks = {1: Stock(id=1, company=companies[1], ticker="AAPL", price=100.0, open_price=100.0, high_price=100.0, low_price=100.0, close_price=100.0, partial_share=0.0, complete_share=0, timestamp=datetime.now(), volatility=0, liquidity=0)}
stock_price_histories = {1: StockPriceHistory(id=1, stock=stocks[1], open_price=100.0, high_price=100.0, low_price=100.0, close_price=100.0, timestamp=datetime.now(), volatility=0, liquidity=0)}
user_profiles = {1: UserProfile(id=1, user_id=1, team_id=1, timestamp=datetime.now())}
orders = {1: Order(id=1, user=user_profiles[1], scenario_id=1, stock=stocks[1], quantity=1, price=100.0, transaction_type="BUY", timestamp=datetime.now())}
transaction_histories = {1: TransactionHistory(id=1, orders=[orders[1]])}
portfolios = {1: Portfolio(id=1, owner=user_profiles[1], balance=float(100.0), stocks=[stocks[1]], transactions=[transaction_histories[1]])}
stock_portfolios = {1: StockPortfolio(id=1, stock=stocks[1], portfolio=portfolios[1], quantity=1)}

# Buy Sell Queue

In [30]:
class BuySellQueue:
    def __init__(self):
        self.buy_queue = deque()
        self.sell_queue = deque()

    def add_to_buy_queue(self, user, asset, amount, price):
        self.buy_queue.append((user, asset, amount, price))

    def add_to_sell_queue(self, user, asset, amount, price):
        self.sell_queue.append((user, asset, amount, price))

    def process_queues(self):
        transactions = []
        while self.buy_queue and self.sell_queue:
            buy_order = self.buy_queue.popleft()
            sell_order = self.sell_queue.popleft()

            if buy_order[3] >= sell_order[3]:  # Buy price >= Sell price
                matched_price = (buy_order[3] + sell_order[3]) / 2
                transactions.append(
                    self.execute_transaction(buy_order, sell_order, matched_price)
                )
        return transactions

    def execute_transaction(self, buy_order, sell_order, price):
        buyer, asset, amount, buy_price = buy_order
        seller, _, sell_amount, sell_price = sell_order

        if amount > sell_amount:
            amount = sell_amount
            self.add_to_buy_queue(buyer, asset, amount - sell_amount, buy_price)

        self.complete_transaction(buyer, seller, asset, amount, price)

        return {
            "buyer": buyer.user_id,
            "seller": seller.user_id,
            "asset": asset.ticker,
            "amount": amount,
            "price": price,
            "timestamp": datetime.now().isoformat(),
        }

    def complete_transaction(self, buyer, seller, asset, amount, price):
        stock = stocks[asset.id]
        stock.price = price
        user_buyer = user_profiles[buyer.id]
        user_seller = user_profiles[seller.id]
        for portfolio in portfolios.values():
            if portfolio.owner.id == user_buyer.id:
                portfolio_buyer = portfolio
                portfolio_buyer.stocks.append(stock)
                portfolio_buyer.balance -= price * amount
            if portfolio.owner.id == user_seller.id:
                portfolio_seller = portfolio
                portfolio_seller.stocks.remove(stock)
                portfolio_seller.balance += price * amount

        print(
            f"Transaction: {buyer.user_id} bought {amount} of {asset.ticker} from {seller.user_id} at {price}"
        )


# Broker

In [ ]:
class Broker:
    def __init__(self, name):
        self.name = name
        self.k = 0.5  # A constant factor representing the market maker's risk tolerance and desired profit margin.
        self.c = 0.01  # A constant representing fixed costs or other market-specific adjustments.
        self.queues: Dict[str, BuySellQueue] = {}

    def get_queue(self, ticker):
        if ticker not in self.queues:
            self.queues[ticker] = BuySellQueue()
        return self.queues[ticker]

    def compute_spread(self, volatility, liquidity):
        spread = self.k * volatility + self.c / liquidity
        return spread

    def adjust_client_price(self, best_bid, best_ask, spread, transaction_type):
        midprice = (best_bid + best_ask) / 2
        if transaction_type == "buy":
            print("Buy")
            print(midprice)
            ask_price = midprice + spread / 2
            return ask_price
        else:
            print("Sell")
            print(midprice)
            bid_price = midprice - spread / 2
            return bid_price

    def get_best_prices(self, asset):
        best_bid = asset.low_price
        best_ask = asset.high_price
        return best_bid, best_ask

    def add_to_buysell_queue(self, user, asset, amount, price, transaction_type):
        best_bid, best_ask = self.get_best_prices(asset)
        current_volume = asset.liquidity
        current_volatility = asset.volatility
        spread = self.compute_spread(current_volatility, current_volume)

        stock_queue = self.get_queue(asset.ticker)

        if transaction_type == "buy":
            adjusted_price = self.adjust_client_price(best_bid, best_ask, spread, "buy")
            stock_queue.add_to_buy_queue(user, asset, amount, adjusted_price)
            print("adjusted_price: ",adjusted_price)
            if adjusted_price > best_ask:
                asset.high_price = adjusted_price
                stocks[asset.id] = asset
                adjusted_price = self.adjust_client_price(
                best_bid, best_ask, spread, "sell"
            )
            stock_queue.add_to_sell_queue(user, asset, amount, adjusted_price)
            print("adjusted_price: ",adjusted_price)
            if adjusted_price < best_bid:
                asset.low_price = adjusted_price
                stocks[asset.id] = asset

    def process_queues(self):
        transactions = []
        for queue in self.queues.values():
            transactions.extend(queue.process_queues())
        return transactions

# Main

In [ ]:
if __name__ == "__main__":
    # Mock data
    user1 = UserProfile(id=1, user_id=1, team_id=None, timestamp=datetime.now())
    user2 = UserProfile(id=2, user_id=2, team_id=None, timestamp=datetime.now())
    company = Company(id=1, name="Test Company")

    user_profiles[user1.id] = user1
    user_profiles[user2.id] = user2
    companies[company.id] = company

    # Portfolio and Stock Setup
    portfolio1 = Portfolio(id=1, owner=user1, balance=1000.00, stocks=[], transactions=[])
    portfolio2 = Portfolio(id=2, owner=user2, balance=1000.00, stocks=[], transactions=[])
    stock = Stock(
        id=1, company=company, ticker="TST", price=100.0, open_price=100.0, high_price=105.0, low_price=95.0, close_price=100.0,
        partial_share=0.0, complete_share=100, timestamp=datetime.now(), volatility=0.1, liquidity=1000.0
    )

    stocks[stock.id] = stock

    # Initialize Broker
    broker = Broker("Algo")

    # Add orders to the queue
    # broker.add_to_buysell_queue(user1, stock, amount=10, price=101.0, transaction_type="buy") # fail
    # broker.add_to_buysell_queue(user1, stock, amount=10, price=99.5, transaction_type="buy") # success
    # broker.add_to_buysell_queue(user2, stock, amount=10, price=99.2, transaction_type="sell") # success
    # broker.add_to_buysell_queue(user2, stock, amount=10, price=101.0, transaction_type="sell") # fail

    # Add orders to the queue 100
    for i in range(100):
        stocks[stock.id] = stock
        broker.add_to_buysell_queue(user1, stock, amount=random.randint(1, 10), price=stock.price + random.uniform(-1, 1), transaction_type="buy")

    # Process the queues
    transactions = broker.process_queues()
    print("Transactions:", transactions)
    print("Stock Price:", stock.price)
    print("Stock High Price:", stock.high_price)
    print("Stock Low Price:", stock.low_price)
    print("Portfolio 1 Balance:", portfolio1.balance)
    print("Portfolio 2 Balance:", portfolio2.balance)

    # Run again to show that queues are empty
    transactions = broker.process_queues()
    print("Transactions:", transactions)
    print("Stock Price:", stock.price)
    print("Stock High Price:", stock.high_price)
    print("Stock Low Price:", stock.low_price)
    print("Portfolio 1 Balance:", portfolio1.balance)
    print("Portfolio 2 Balance:", portfolio2.balance)

[![](https://mermaid.ink/img/pako:eNq1k7FuwjAQhl_l5KULFbsHpFaV2qUSlLJ5MfYBbhI7nO0hQrx7nYQ0IEPL0gxR7Pvv93eX84Epp5Fx5nEf0Sp8MXJLshIW0lNLCkaZWtoAK480J7cxJebBZ3IFUr6_DE4VV-SxWWJZLiLGK2ZzR2HjSuPy0CdJ66UKxtk344OjRthedYYHj7PZiYjDvJQKYR0bcKQHxD7Y6TpCDq8YYI0-vYyeSl9ATUahz-SD7ZP-iq06-XbKXHhWYivXIwMEB_u-9HvYfbL5F_jO-F76kSLDv5mXakoYvlcPOGeKC6h3GdSubdJ0PCov4Wc2OKxqLUP3a5EePNSXU_NrTnvCH0mn1g4J7epGr_KZ5PCBKhUAYQwJyyasQqqk0em6HVobwcIOKxSMp08tqRBM2GPSyRjcsrGK8UARJ4xc3O4Y38jSp1XsmE4XtZccvwHZckra?type=png)](https://mermaid.live/edit#pako:eNq1k7FuwjAQhl_l5KULFbsHpFaV2qUSlLJ5MfYBbhI7nO0hQrx7nYQ0IEPL0gxR7Pvv93eX84Epp5Fx5nEf0Sp8MXJLshIW0lNLCkaZWtoAK480J7cxJebBZ3IFUr6_DE4VV-SxWWJZLiLGK2ZzR2HjSuPy0CdJ66UKxtk344OjRthedYYHj7PZiYjDvJQKYR0bcKQHxD7Y6TpCDq8YYI0-vYyeSl9ATUahz-SD7ZP-iq06-XbKXHhWYivXIwMEB_u-9HvYfbL5F_jO-F76kSLDv5mXakoYvlcPOGeKC6h3GdSubdJ0PCov4Wc2OKxqLUP3a5EePNSXU_NrTnvCH0mn1g4J7epGr_KZ5PCBKhUAYQwJyyasQqqk0em6HVobwcIOKxSMp08tqRBM2GPSyRjcsrGK8UARJ4xc3O4Y38jSp1XsmE4XtZccvwHZckra)
            print("adjusted_price: ",adjusted_price)
            stock_queue.add_to_buy_queue(user, asset, amount, adjusted_price)
            print("adjusted_price: ",adjusted_price)
            stock_queue.add_to_sell_queue(user, asset, amount, adjusted_price)

[![](https://mermaid.ink/img/pako:eNqdVMtu2zAQ_JUFL724P8BDgCYtkB7aGnFz04UmVzZhiasulwcjyL-XesSWJaVQqpO0M7Pcx4gvypJDpVXEPwmDxa_eHNjURYD8NIbFW9-YIPAckbdMpa9wDt4znZDn8R-GTyjz-C92yPdEpyWJ2KMPh2_h4MPCUQ8VGs74I6W4AG-JpaTK0xzaCdmFA3-zCdFY8RQefRTicxF61qhl-Hx3N3SpYVsZi7BPZ6C2j57cgx2v71rDLu1rL1Nij3bEyxg0fHFuSrygQ9LxXDQ8HNGeoCSGukXW1RyxqtYVPWW-X_WU-V9lm2owRXyLw_DcJuhy3lggt8ZkMUaQ6yKv8htup744RMNz44x0m0T-FKG59c5aeTuA9frOhVdt-wUNe4v_Es09quEJbZ76ctcLQ3szwk8SX57B7CnJsnhkipGbNHwPeW31e0qscq3DCh1BIFmxyZGVnlCMD5BCp0LXWyp-vKdJhg_2tajG4IqgNqpGro13-bp8aYFCyRFrLJTOry7_IIUqwmvmmSS0OwertHDCjWJKh6PSpckz2qjUrX64aIfo619gqeAP?type=png)](https://mermaid.live/edit#pako:eNqdVMtu2zAQ_JUFL724P8BDgCYtkB7aGnFz04UmVzZhiasulwcjyL-XesSWJaVQqpO0M7Pcx4gvypJDpVXEPwmDxa_eHNjURYD8NIbFW9-YIPAckbdMpa9wDt4znZDn8R-GTyjz-C92yPdEpyWJ2KMPh2_h4MPCUQ8VGs74I6W4AG-JpaTK0xzaCdmFA3-zCdFY8RQefRTicxF61qhl-Hx3N3SpYVsZi7BPZ6C2j57cgx2v71rDLu1rL1Nij3bEyxg0fHFuSrygQ9LxXDQ8HNGeoCSGukXW1RyxqtYVPWW-X_WU-V9lm2owRXyLw_DcJuhy3lggt8ZkMUaQ6yKv8htup744RMNz44x0m0T-FKG59c5aeTuA9frOhVdt-wUNe4v_Es09quEJbZ76ctcLQ3szwk8SX57B7CnJsnhkipGbNHwPeW31e0qscq3DCh1BIFmxyZGVnlCMD5BCp0LXWyp-vKdJhg_2tajG4IqgNqpGro13-bp8aYFCyRFrLJTOry7_IIUqwmvmmSS0OwertHDCjWJKh6PSpckz2qjUrX64aIfo619gqeAP)


        
        stocstock.s[]stock.id